#### Декораторы

В python функции - `first class citizens`. Это значит, что они могут быть переменными, аргументами и возвращаемыми значениями других функций.



In [5]:
def my_function():
    print("Hello, world")

other_function = my_function

other_function()

Hello, world


In [19]:
%load_ext nb_mypy

from typing import Any, Callable
def function_creator(func: Callable[..., Any]) -> Callable[..., Any]:
    """
    Принимаем на вход функцию, а возвращаем другую функцию, которую сами только что сделали
    Может иметь отношение к исходной, а может и нет (порицается, но не исключено)
    """
    def function_wrapper(*args, **kwargs):
        print("Gotcha")
        return func(*args, **kwargs)
    
    return function_wrapper    

def f(x: int, y: int) -> int:
    return x + y

print(f(1, 2))

g = function_creator(f)

print(g(2, 3))
f = g
print(f(3, 4))

The nb_mypy extension is already loaded. To reload it, use:
  %reload_ext nb_mypy
3
Gotcha
5
Gotcha
7


То же самое можно записать короче: выражение вида
```python
f = function_creator(f)
```
Удобно записывается так


In [12]:
def function_creator(func: Callable[..., Any]) -> Callable[..., Any]:
    def function_wrapper(*args, **kwargs):
        print("Gotcha")
        return func(*args, **kwargs)
    
    return function_wrapper

@function_creator
def my_function(a, b):
    return a + b

print(my_function(1, 2))  

Gotcha
3


И вот функция (строго говоря, может быть и класс, в общем случае `Callable`), которая модифицирует переданную в нее функцию, называется **декоратор**

Зачем это нужно:
Привнести какую-то логику, паралельную бизнес-логике декорируемого `Callable`
  * Логирование переданных параметров и результата
  * Авторизация/Проверка нужных разрешений
  * Повторное выполнение каких-то запросов в случае возникновения временных ошибок, когда безопасно пробовать еще раз
  * Кэширование ранее сделанных вычислений (мемоизация)
  * Связь `Callable` и когда их нужно вызывать: веб фреймворки и всякие библиотеки для ботов
  * Конструкции самого `python`: `@classmethod`, `@staticmethod`, `@property`, и другие

In [21]:
# Правила хорошего тона
from typing import Any, Callable
import functools
def my_decorator(func: Callable[[Any], Any]):
    # внутреннюю функцию называем `wrapper`
    # сигнатура у нее в общем случае `def wrapper(*args, **kwargs)`, потому что неизвестно, какая будет сигнатура у
    # декорируемой функции. 
    
    # @functools.wraps(func) нужно для того, чтобы не потерять информацию о декорируемой функции:
    # имя, документация, модуль и другие
    @functools.wraps(func)
    # крайне порицается добавление новых аргументов, которых не было в исходной.
    # extra_arg добавлять не надо!
    # Почему? потому что если вдруг появляется функция с тем же именем,
    #  то ожидается, что и вести себя она будет так же, как исходная
    def wrapper(*args, extra_arg=None, **kwargs) -> Any:
        print("Before")
        # здесь гипотетически мы можем вызвать декорируемую функцию не с переданными аргументами,
        # а с произвольными. Делать этого чаще всего не нужно и даже вредно
        func(*args, **kwargs)
        print("After")
    return wrapper

@my_decorator
def f(x):
    print(x)

f(10)

Before
10
After


В примере выше декоратор менял исходную функцию всегда одинаково и независимо ни от чего. Но что, если есть необходимость управлять этим поведением? 

In [ ]:

import functools

# здесь аргументы `retry_decorator` - параметры самого декоратора 
def retry_decorator(attempts: int):
    def decorator(func):
        # помним про волшебную функцию `wraps`
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            # здесь есть доступ и к аргументам `retry_decorator`, и к аргументам `decorator` 
            # и к аргументам `func`
            for _ in range(attempts):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    print(f"Exception occurred: {e}")
            raise Exception("Maximum attempts reached")
        return wrapper
    # именно эту функцию вернем, а она, будучи вызванной с нужными аргументами, 
    # вернет подготовленный декоратор, который уже модифицирует исходную функцию
    return decorator
    
@retry_decorator(3)
def my_function():
    print("Hello, world")

@retry_decorator # так работать уже не будет
def my_function_with_args(arg1, arg2):
    print(f"Hello, {arg1} and {arg2}")

# здесь my_function_with_args превратилась в функцию, которая ждет на вход функцию, 
# чтобы далее ее задекорировать, а получает на вход содержательные аргументы
my_function_with_args("John", "Doe")

TypeError: retry_decorator.<locals>.decorator() takes 1 positional argument but 2 were given

##### Задание 1 - Логирующий декоратор
Необходимо написать декоратор, который логирует (выводит на экран) название функции, переданные в нее аргументы (args и kwargs) и возвращаемое значение. Вывод должен состояться в любом случае, даже если декорируемая функция выбрасывает исключение. (вам *возможно* понадобится блок `finally` из try/except)

##### Задание 2 - Параметризованный volkswagen-test декоратор
```python
def test_decorator(test_mode: bool = False):
    # если test_mode is True - игнорировать любые исключения, которые могут быть выброшены внутри декорируемой функции
    # если test_mode is False - полностью сохранить поведение исходной функции
    pass    

```